# ISS623: Applied Healthcare Analytics

# Lecture on Prescriptive Analytics - Schedule Optimization

Workshop Instructions: 

1. Read through the text descriptions at the top of the cell/code blocks
2. Run the code below by selecting the code block and pressing ``Ctrl + Enter``. Note: The preceding code blocks must be run before proceeding to the next block.
2. Think through the guiding questions and points that are raised for the step. What do you observe from the output and what do they mean?

This workshop on schedule assignment uses the `ortools` package from Google OR-Tools: https://developers.google.com/optimization


In [1]:
# run this if you need to install the ortools package
!pip install ortools


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np

# Simple Crew Scheduling Problem

**Minimising Costs for COVID-19 Manpower Deployment in External Isolation Facilities**

- TechHealth has been given an assignment to deploy trained nurses in external isolation facilities to deal with the surge in COVID-19 infections in the community
- It need to hire additional trained nurses to cover shifts necessary for these facilities
- Integer Programming model can be set up to estimate the number of additional staff to hire to minimize additional costs


## Part 1: Shifts Defined by Management

### 1. Set up CpModel object

The `CpModel()` object represents the whole model that will hold all the decision variables, constraints and objective function. 

In [3]:
model = cp_model.CpModel()

### 2. Define Decision Variables

Define the decision variables for each of the shifts below. Note that for each of variables, a lower and upper limit must be set. These limits may have an effect on the feasibility of the solution and the optimal solution so they need to be set carefully. If the variable can be set to a value of 0, remember to set the lower limit to 0 as well.

$x_1$: Number of hires for Shift 1 (6AM – 2PM);

$x_2$: Number of hires for Shift 2 (8AM – 4 PM);

$x_3$: Number of hires for Shift 3 (12Noon – 8PM); 

$x_4$: Number of hires for Shift 3 Shift 4 (4PM – 12 MN);

$x_5$: Number of hires for Shift 3 Shift 5 (12MN – 6AM)

In [4]:
x_1 = model.NewIntVar(1,1000,'x_1')
x_2 = model.NewIntVar(1,1000,'x_2')
x_3 = model.NewIntVar(1,1000,'x_3')
x_4 = model.NewIntVar(1,1000,'x_4')
x_5 = model.NewIntVar(1,1000,'x_5')

### 3. Add model constraints

Define the constraints in the model below. In `ortools`, the constraints are set in the same manner as you would for a mathematical model. The variables and equality operators e.g. `>=`, `==`, `+` all can be used with the `model.Add()` method.

1. $x_{1}\geq 48$
2. $x_{1} + x_{2} \geq 79$
3. $x_{1} + x_{2} + x_{3} \geq 88$
4. $x_{2} + x_{3} \geq 64$
5. $x_{3} + x_{4} \geq 89$
6. $x_{4} \geq 43$
7. $x_{4} + x_{5} \geq 55$
8. $x_{5} \geq 16$

In [5]:
model.Add(x_1 >= 48)
model.Add(x_1 + x_2 >= 79)
model.Add(x_1 + x_2 + x_3 >= 88)
model.Add(x_2 + x_3 >= 64)
model.Add(x_3 + x_4 >= 89)
model.Add(x_4 >= 43)
model.Add(x_4 + x_5 >=55)
model.Add(x_5 >= 16)
for x in [x_1,x_2,x_3,x_4,x_5]:
    model.Add(x >= 0)

### 4. Define Objective Function:

The objective function here is to minimize the total cost of the hires and can be defined as follows

$Min  Z = 170x_1 + 180x_2 + 156x_3 + 180x_4 + 205x_5$

The objective function equation will have to be used with the `model.Minimize()` method to define the objective in the model object.


In [6]:
model.Minimize( 170*x_1 + 180*x_2 + 156*x_3 + 180*x_4 + 205*x_5)

### 5. Select Solver and Solve Model

The default solver used for most models is the `CpSolver` solver, but there are also others than you can explore in the Google OR-Tools documentation. After running the code below, you should see an 'OPTIMAL' output printed. This indicates that the model is feasible and has reached an optimal solution. At times, if the model is defined incorrectly or is infeasible, the 'INFEASIBLE' may also appear and the model defined above has to be checked for errors. 

In [7]:
solver = cp_model.CpSolver()
solver.Solve(model)
print(solver.StatusName())

OPTIMAL


### 6. Display optimal solution and objective value (total cost)

The optimal solution can be displayed using `solver.Value()` and passing the variable name into the method. At the same time, th objective value can also be displayed using `solver.ObjectiveValue()`. Run the code below and look at the output. Is this reasonable according to what was defined in the model?

In [8]:
for x in [x_1,x_2,x_3,x_4,x_5]:
    print('Hires for ' + x.Name() +': ' + str(solver.Value(x)))

print('Total cost of hires: ' + str(solver.ObjectiveValue()))

Hires for x_1: 61
Hires for x_2: 18
Hires for x_3: 46
Hires for x_4: 43
Hires for x_5: 16
Total cost of hires: 31806.0
